In [2]:
from dask.distributed import Client, LocalCluster
from dask_jobqueue import PBSCluster
from distributed.utils import tmpfile
import os

/apps/jupyterlab/3.4.3-py3.9/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile
/jobfs/91147399.gadi-pbs/ipykernel_2506975/2795989920.py:3: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [4]:
os.environ['DASK_PYTHON'] = '/scratch/vp91/Training/python3.9-venv/bin/python'
setup_commands = ["module load python3/3.9.2", "source /scratch/vp91/Training/python3.9-venv/bin/activate"]
extra = ['-q normal','-P vp91', '-l ncpus=48', '-l mem=192GB']

In [33]:
cluster = PBSCluster(walltime="00:50:00", 
                     cores=48, 
                     memory="192GB",
                     #processes=48, 
                     shebang='#!/usr/bin/env bash',
                     job_extra=extra, 
                     local_directory='$TMPDIR', 
                     header_skip=["select"], 
                     interface="ib0",
                     env_extra=setup_commands,
                     python=os.environ["DASK_PYTHON"])

/apps/jupyterlab/3.4.3-py3.9/lib/python3.9/site-packages/distributed/node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44117 instead
  warnings.warn(


In [ ]:
os.environ['DASK_PYTHON'] = '/scratch/vp91/Training/environment/bin/python'
setup_commands = ["module load python3/3.9.2", "source /scratch/vp91/Training/environment/bin/activate"]
extra = ['-q normal',
         '-P vp91', 
         '-l ncpus=48', 
         '-l mem=192GB']

cluster = PBSCluster(walltime="00:50:00", 
                     cores=48, 
                     memory="192GB",
                     #processes=48, 
                     shebang='#!/usr/bin/env bash',
                     job_extra=extra, 
                     local_directory='$TMPDIR', 
                     header_skip=["select"], 
                     interface="ib0",
                     env_extra=setup_commands,
                     python=os.environ["DASK_PYTHON"])

In [34]:
print(cluster.job_script())

#!/usr/bin/env bash

#PBS -N dask-worker
#PBS -l walltime=00:50:00
#PBS -q normal
#PBS -P vp91
#PBS -l ncpus=48
#PBS -l mem=192GB
module load python3/3.9.2
source /scratch/vp91/Training/python3.9-venv/bin/activate
/scratch/vp91/Training/python3.9-venv/bin/python -m distributed.cli.dask_worker tcp://10.6.41.71:36145 --nthreads 6 --nprocs 8 --memory-limit 22.35GiB --name dummy-name --nanny --death-timeout 60 --local-directory $TMPDIR --interface ib0 --protocol tcp://



In [27]:
cluster.scale(2)

In [28]:
cluster

Dashboard: /proxy/39043/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.41.71:44305,Workers: 0
Dashboard: /proxy/39043/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [29]:
client = Client(cluster)

In [30]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: /proxy/39043/status,
Dashboard: /proxy/39043/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.41.71:44305,Workers: 0
Dashboard: /proxy/39043/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [31]:
!which python

/apps/jupyterlab/3.4.3-py3.9/bin/python


In [32]:
!qstat

Job id                 Name             User              Time Use S Queue
---------------------  ---------------- ----------------  -------- - -----
91116294.gadi-pbs      sys-dashboard-s* jj8451            00:00:45 R normal-exec     
91117718.gadi-pbs      dask-worker      jj8451                   0 Q normal-exec     


In [24]:
!pwd


/home/659/jj8451


In [25]:
!pwd

/home/659/jj8451


In [26]:
cluster1 = LocalCluster()


/apps/jupyterlab/3.4.3-py3.9/lib/python3.9/site-packages/distributed/node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42961 instead
  warnings.warn(


In [22]:
cluster1

LocalCluster(b472c28f, 'tcp://127.0.0.1:42395', workers=8, threads=48, memory=188.57 GiB)

In [1]:
def slow_increment(x):
    time.sleep(10)
    return x+1

In [ ]:
futures = client.submit(slow_increment, 5000)